In [2]:
# import pandas as pd
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import control as ct

In [4]:
%matplotlib

Using matplotlib backend: <object object at 0x000001BECFC40EE0>


In [3]:
#model response
m = 0.06
R = 0.0175
g = 9.8
L = 0.20
d = 0.12
J = (2/5)*m*R**2
s = ct.tf('s')
G = ((m*g*d)/(L*((J/R**2)+m)))*(1/s**2)

Kp = 3
Ki = 0.3
Kd = 4

C = Kp+Ki/s+Kd*s
print(C)
plant = ct.feedback(G*C)
T,U = ct.step_response(15*plant,T = 3)
fig8,ax8 = plt.subplots()

L = len(U)
zeros = []
for i in range(0,L-1):
    if abs((U[-1]-U[i-1])/U[-1])>0.05 and abs((U[-1]-U[i+1])/U[-1])<0.05:
        zeros.append(T[i])
for i in range(0,L):
    if (U[-1]-U[i])/U[-1]<0.05 and T[i]>=zeros[-1]:
        I = i
        sett=T[i]
        Usett = U[i]
        break

dict = ct.step_info(15*plant)
print(dict)
over = dict['Overshoot']
print('Overshoot = ',over,'%')
rise = dict['RiseTime']
print('Rise time =',rise,'s')
print("Settling time =",sett,'s')

ax8.plot(T,U,label = 'response')
ax8.axhline(15,color = 'red',linestyle ='dotted',label = 'setpoint')
ax8.axhline(U[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
ax8.axhline(U[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax8.grid()
ax8.plot(sett,Usett,'*',label = "settling point",color = 'r')
#ax8.vlines(0.5,ymin=15,ymax = U[-1],color='r',label = "steady state error")
ax8.vlines(T[np.argmax(U)],ymin=15,ymax=np.max(U),label = 'Overshoot', color = 'red')
ax8.legend()
ax8.set_ylabel('Displacement [cm]')
ax8.set_xlabel('Time [s]')
fig8.suptitle('Trial-and-Error Tuned Model Step Response for Squash Ball')


4 s^2 + 3 s + 0.3
-----------------
        s

{'RiseTime': 0.11761123112821635, 'SettlingTime': nan, 'SettlingMin': 13.617478977783572, 'SettlingMax': 15.55583258746147, 'Overshoot': 3.7055505830764806, 'Undershoot': 0, 'Peak': 15.55583258746147, 'PeakTime': 0.400749380140589, 'SteadyStateValue': 14.999999999999998}
Overshoot =  3.7055505830764806 %
Rise time = 0.11761123112821635 s
Settling time = 0.15517241379310345 s


Text(0.5, 0.98, 'Trial-and-Error Tuned Model Step Response for Squash Ball')

In [17]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(5,1,figsize=(6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)
figraw,axsraw = plt.subplots(5,1,figsize=(6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)
figz,axz = plt.subplots(5,1,figsize = (6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)

fig.suptitle('Individual Trial-And-Error Tuned Responses for Marble')
figraw.suptitle('Unfiltered Trial-And-Error Tuned Responses for Marble')
figz.suptitle('Motor Angle for Trial-And-Error Tuned Marble Response')

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of Trial-And-Error Tuned Marble Responses')
#fig3.suptitle('Unfiltered Average of Manually Tuned Marble Responses')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Distance [cm]')
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(4, 1000, 'lp', fs=50000, output='sos')

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 1422
for i in range(1,6):
    df = pd.read_csv('MARBLE_MAN_{}.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs[i-1].plot(timenew,filtered,label = 'filtered',color = 'b')
    axs[i-1].plot(timenew,distance,alpha=0.4,label = 'unfiltered',color = 'darkslategrey')
    axs[i-1].set_xlabel('Time [s]')
    axs[i-1].set_ylabel('Distance [cm]')
    axs[i-1].axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
    axs[i-1].grid()
    
    axsraw[i-1].plot(timenew,distance)
    axsraw[i-1].set_xlabel('Time [s]')
    axsraw[i-1].set_ylabel('Distance [cm]')
    axsraw[i-1].grid()
    
    axz[i-1].plot(timenew,angle_filt,label = 'filtered',color = 'b')
    axz[i-1].plot(timenew,angle,alpha=0.35,label = 'unfiltered',color = 'darkslategrey')
    axz[i-1].set_xlabel('Time [s]')
    axz[i-1].set_ylabel('Servo Angle [degrees]')
    axz[i-1].grid()
    axz[i-1].axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()
    
axs[0].legend(loc = 'lower right')
axz[0].legend(loc = 'lower right')
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)
#print(time_ave)
filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
for i in range(0,L-1):
    if time_ave[i]<2.5 and filtered[i]>filtered[i-1]:
        OS = filtered[i]
        OST = time_ave[i]
# OS = np.max(filtered)
# OSind = np.argmax(filtered)
# OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right')
ticks = np.linspace(0,30,31)
ax2.set_yticks(ticks)
#ax3.plot(time_ave,dist_ave,label = 'response')

1554
1451
1460
1425
1422


In [13]:
# fig,axs = plt.subplots(5,1,figsize=(6,9))
# plt.subplots_adjust(top = 0.95,bottom = 0.05)
# figraw,axsraw = plt.subplots(5,1,figsize=(6,9))
# plt.subplots_adjust(top = 0.95,bottom = 0.05)

# fig.suptitle('Filtered Manually Tuned Responses for Marble')
# figraw.suptitle('Unfiltered Manually Tuned Responses for Marble')

# fig2,ax2 = plt.subplots()
# fig3,ax3 = plt.subplots()
# fig2.suptitle('Filtered Average of Manually Tuned Marble Responses')
# fig3.suptitle('Unfiltered Average of Manually Tuned Marble Responses')
# ax2.set_xlabel('Time [s]')
# ax2.set_ylabel('Distance [cm]')
# ax3.set_xlabel('Time [s]')
# ax3.set_ylabel('Distance [cm]')

# time_1 = []
# time_2 = []
# time_3 = []
# time_4 = []
# time_5 = []
# distance_1 = []
# distance_2 = []
# distance_3 = []
# distance_4 = []
# distance_5 = []
# time_ave = []
# dist_ave = []

# sos = signal.butter(3, 1000, 'lp', fs=50000, output='sos')

# for i in range(1,6):
#     df = pd.read_csv('MARBLE_MAN_{}.csv'.format(i))
    
#     time = (df['time']-5000)/1000
#     distance = df['distance']
#     filtered = signal.sosfiltfilt(sos,distance)
    
#     axs[i-1].plot(time,filtered)
#     axs[i-1].set_xlabel('Time [s]')
#     axs[i-1].set_ylabel('Distance [cm]')
    
#     axsraw[i-1].plot(time,distance)
#     axsraw[i-1].set_xlabel('Time [s]')
#     axsraw[i-1].set_ylabel('Distance [cm]')
    
#     time_name =f"time_{i}"
#     globals()[time_name]=((time[0:1363].to_numpy()))
#     distance_name = f"distance_{i}"
#     globals()[distance_name]=distance[0:1363].to_numpy()

# for i in range(0,1363):
#     time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
#     dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)

# filtered = signal.sosfiltfilt(sos,dist_ave)
# ax2.plot(time_ave,filtered)
# ax3.plot(time_ave,dist_ave)

In [6]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)
figraw,axsraw = plt.subplots()

figz,axz = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)

fig.suptitle('Individual Trial-And-Error Tuned Responses for Marble',fontsize = 15)
figraw.suptitle('Unfiltered Trial-And-Error Tuned Responses for Marble',fontsize = 15)
figz.suptitle('Motor Angle for Trial-And-Error Tuned Marble Response',fontsize = 15)

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of Trial-And-Error Tuned Marble Responses',fontsize = 15)
#fig3.suptitle('Unfiltered Average of Manually Tuned Marble Responses')
ax2.set_xlabel('Time [s]',fontsize = 15)
ax2.set_ylabel('Distance [cm]',fontsize = 15)
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(4, 1000, 'lp', fs=50000, output='sos')
lines = ['solid','dashed','dotted','dashdot','solid']

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 1422
ticksang = np.linspace(-30,30,11)
for i in range(1,6):
    df = pd.read_csv('MARBLE_MAN_{}.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs.plot(timenew,filtered,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axs.set_xlabel('Time [s]')
    axs.set_ylabel('Distance [cm]')
    
    axs.grid()
    
    axsraw.plot(timenew,distance)
    axsraw.set_xlabel('Time [s]',fontsize = 15)
    axsraw.set_ylabel('Distance [cm]',fontsize = 15)
    axsraw.grid()
    
    axz.plot(timenew,angle_filt,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axz.set_xlabel('Time [s]',fontsize = 15)
    axz.set_ylabel('Servo Angle [degrees]',fontsize = 15)
    axz.grid()

    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()

    
    
    
axs.axhline(15,color = 'r',linestyle = 'dotted',label = 'Setpoint')    
axz.axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    
axs.legend(loc = 'lower right')
axz.legend(loc = 'lower right')
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)
print(time_ave)
filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
for i in range(0,L-1):
    if time_ave[i]<1 and filtered[i]>filtered[i-1]:
        OS = filtered[i]
        OST = time_ave[i]
# OS = np.max(filtered)
# OSind = np.argmax(filtered)
# OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right')
ticks = np.linspace(0,25,26)
ax2.set_yticks(ticks)

#ax3.plot(time_ave,dist_ave,label = 'response')

1554
1451
1460
1425
1422
[0.014000000000000002, 0.031, 0.049, 0.066, 0.084, 0.101, 0.118, 0.136, 0.1532, 0.172, 0.189, 0.207, 0.2242, 0.24139999999999998, 0.25939999999999996, 0.27640000000000003, 0.295, 0.3122, 0.3294, 0.3474, 0.36479999999999996, 0.38280000000000003, 0.4, 0.4178, 0.4364, 0.45380000000000004, 0.47359999999999997, 0.491, 0.5094000000000001, 0.5267999999999999, 0.5446000000000001, 0.5624, 0.5795999999999999, 0.5982000000000001, 0.6155999999999999, 0.6334, 0.651, 0.6686, 0.6866000000000001, 0.7038, 0.7224, 0.7398, 0.7576, 0.776, 0.7938000000000001, 0.8122, 0.8300000000000001, 0.849, 0.867, 0.885, 0.9044000000000001, 0.923, 0.942, 0.96, 0.9790000000000001, 0.9972, 1.0162, 1.0354, 1.054, 1.0734000000000001, 1.092, 1.1114000000000002, 1.1298, 1.1488, 1.1666, 1.1851999999999998, 1.2038, 1.2218, 1.2408000000000001, 1.2588, 1.2778, 1.2958, 1.3144000000000002, 1.333, 1.3512, 1.3708000000000002, 1.3887999999999998, 1.4083999999999999, 1.4265999999999999, 1.446, 1.464600000000000